In [ ]:
# packages
import os
import torch
import pandas as pd
# from transformers import BertModel, DistilBertModel
from tqdm import tqdm
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
# from transformers import BertForSequenceClassification

In [ ]:
# import data
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name = "merged_data_set_AIR.xlsx"
data_set = pd.read_excel(os.path.join(data_path, file_name))
data_set.columns = ['no',
                    'uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']
data_set = data_set.reset_index()
data_set['content'] = data_set['content'].astype(str)
data_set.head(1)

In [ ]:
data_set = data_set.dropna(subset = ["importance"])
data_set = data_set.reset_index(drop = True)
data_set['index'] = data_set.index
data_set.shape

In [ ]:
data_set.head(1)

In [ ]:
data_set.tail(1)

In [ ]:
# encoding the labels
possible_labels = data_set.importance.unique()
encoded_values = [1, 0, 2]

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = encoded_values[index]
label_dict

In [ ]:
data_set['label'] = data_set.importance.replace(label_dict)

In [ ]:
data_set.index.values

In [ ]:
data_set.label.values

In [ ]:
# train and validation split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data_set.index.values,
                                                 data_set.label.values,
                                                 test_size = 0.2,
                                                 random_state = 42,
                                                 stratify = data_set.label.values)
data_set['data_type'] = ['not_set'] * data_set.shape[0]

In [ ]:
data_set.loc[X_train, 'data_type'] = 'train'
data_set.loc[X_val, 'data_type'] = 'val'

data_set.groupby(['importance', 'label', 'data_type']).count()

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)